# Evaluate RAG

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService

provider = ProviderService()

In [2]:
QA_REPO = "BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong"
QA_SPLIT = "INDEX.medium_index_TDT"
qa_dataset = load_dataset(QA_REPO, split=QA_SPLIT)
qa_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
    num_rows: 144
})

# Run RAG

In [7]:
# donwload corpora dataset
DATA_REPO = "BroDeadlines/TEST.TDT.edu_tdt_proposition_data"
DATA_SPLIT = "INDEX.medium_index_TDT_clean"

dataset = load_dataset(DATA_REPO, split=DATA_SPLIT)
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [11]:
from src.rag.hg_parent_retriever import HugFaceParentRAG, get_default_config

config = get_default_config()

config['vec_index'] = "vec-sentence-propositon_medium_edu_tdt"
config['txt_index'] = "text-sentence-propositon_medium_edu_tdt"

config

{'llm': 'gemini-1.5-flash',
 'total_k': 4,
 'txt_weight': 0.5,
 'vec_weight': 0.5,
 'vec_index': 'vec-sentence-propositon_medium_edu_tdt',
 'txt_index': 'text-sentence-propositon_medium_edu_tdt'}

In [8]:
rag = HugFaceParentRAG(provider=provider, config=config, text_corpora=dataset)

## Run

In [3]:
a = "Điều kiện để được xét miễn các học phần Anh văn giao tiếp là như thế nào"
b = "Những lưu ý đối với kỳ thi chứng chỉ quốc tế sau khi kết thúc học phần tiếng Anh cuối cùng trong chương trình học là gì"

a = [a, b]
"?\n".join(a)

'Điều kiện để được xét miễn các học phần Anh văn giao tiếp là như thế nào?\nNhững lưu ý đối với kỳ thi chứng chỉ quốc tế sau khi kết thúc học phần tiếng Anh cuối cùng trong chương trình học là gì'

In [1]:
def eval_no_score(row):
    resp = rag.answer(row['question'])
    row['answer'] = resp['answer']
    row['exc_second'] = resp['exc_second']
    docs = resp['retrieved_docs']
    result = [{'content': d.page_content, "doc_id": d.metadata['doc_id'], "score": -1.0} for d in docs]
    row['evaluation'] = result
    return row

class MultiDocEval:
    
    def __init__(self, rag):
        self.questions = []
        self.rag = rag
        return

    def eval_multi_doc(self, row, n_doc:int = 2, overlap = False):
        ques = row['question'] 
        while len(self.questions) != n_doc:
            self.questions.append(ques)
        resp = self.rag.answer("?\n".join())
        row['answer'] = resp['answer']
        row['exc_second'] = resp['exc_second']
        docs = resp['retrieved_docs']
        result = [{'content': d.page_content, "doc_id": d.metadata['doc_id'], "score": -1.0} for d in docs]
        row['evaluation'] = result
        return row

In [10]:
eval_dataset = qa_dataset.map(eval_no_score)

Parameter 'function'=<function eval_no_score at 0x7f6f89f91040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

page_content='Khi thí sinh hoàn tất các thủ tục nhập học như đóng lệ phí, nộp hồ sơ… thí sinh sẽ chính thức là sinh viên của trường.' metadata={'doc_id': 'ebe592de-13cc-11ef-b548-0242ac1c000c'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Điều khiển các thiết bị và hệ thống tự động' metadata={'doc_id': 'ebe5a04e-13cc-11ef-b548-0242ac1c000c'}


...

page_content='Thí sinh đăng ký xét tuyển vào ngành "Dược học" phải cập nhật học lực lớp 12 từ loại "Giỏi" trở lên mới đủ điều kiện xét tuyển.' metadata={'doc_id': 'ebe866bc-13cc-11ef-b548-0242ac1c000c'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Theo công văn hỗ trợ con em CBCĐ, mỗi tháng sẽ hỗ trợ 100.000đ cho sinh viên là con em CBCĐ.' metadata={'doc_id': '573ca209-1b6d-11ef-a755-d38426455a06'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Điểm không nhân hệ số, đã bao gồm điểm ưu tiên khu vực và ưu tiên đối tượng' metadata={'doc_id': 'ebe71618-13cc-11ef-b548-0242ac1c000c'}
page_content='Các môn học chuyên ngành được chọn lựa dựa theo nhu cầu của thị trường lao động' metadata={'doc_id': 'ebe5b87c-13cc-11ef-b548-0242ac1c000c'}
page_content='Ký túc xá hỗ trợ cho phụ huynh lên thăm chỗ ăn ở sinh hoạt của sinh viên.' metadata={'doc_id': '573ca1ad-1b6d-11ef-a755-d38426455a06'}
page_content='Những môn học được quy định không tham gia tính điểm trung bình sẽ không đưa vào công thức này để tính' metadata={'doc_id': '573ca1c3-1b6d-11ef-a755-d38426455a06'}
page_content='Ký túc xá (KTX) chỉ hỗ trợ chỗ ở nội trú cho sinh viên đang học tại trường.' metadata={'doc_id': '573ca19e-1b6d-11ef-a755-d38426455a06'}
page_content='Các môn học chuyên ngành được chọn lựa dựa theo nhu cầu của thị trường lao động;' metadata={'doc_id': 'ebe5a04e-13cc-11ef-b548-0242ac1c000c'}
page_content='Sinh viên được hưởng nhiều chính sách ưu đãi v

## Push

In [13]:
eval_dataset.push_to_hub("BroDeadlines/EVAL.RAG.evaluation", split="tdt_propostion_parent_k4")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/680 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.RAG.evaluation/commit/8f625288677917ced4e9a728f25a12e7bf438265', commit_message='Upload dataset', commit_description='', oid='8f625288677917ced4e9a728f25a12e7bf438265', pr_url=None, pr_revision=None, pr_num=None)

# Report

In [4]:
eval_dataset = load_dataset("BroDeadlines/EVAL.RAG.evaluation", split="tdt_propostion_parent_k4")

In [5]:
from src.utils.eval_utils import evaluate_IR, calculate_BLEU, calculate_ROUGE

preds = eval_dataset['answer']
grounds = qa_dataset['answer']
len(preds) == len(grounds)

bleu = calculate_BLEU(grounds=grounds, preds=preds)
# bleu
rouge = calculate_ROUGE(grounds=grounds, preds=preds)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using low

In [6]:
data = {}
t_from = 6
t_to = 8

for i in range(t_from, t_to+1):
    res = evaluate_IR(eval_dataset=eval_dataset, limit_k=i)
    del res['relevant']
    data[f'k_{i}'] = res
data

{'k_6': {'precision': 0.6510416666666666,
  'recall': 0.8680555555555556,
  'map_score': 0.2829166666666666,
  'relevant_retrieved': 125,
  'num_retrieved': 192},
 'k_7': {'precision': 0.6386138613861386,
  'recall': 0.8958333333333334,
  'map_score': 0.28290426587301587,
  'relevant_retrieved': 129,
  'num_retrieved': 202},
 'k_8': {'precision': 0.6650485436893204,
  'recall': 0.9513888888888888,
  'map_score': 0.2726467427248678,
  'relevant_retrieved': 137,
  'num_retrieved': 206}}

In [9]:
import numpy as np
time = eval_dataset['exc_second']
time_np = np.array(time)

In [31]:
es = provider.get_elasticsearch_store(index=config['vec_index'])

In [13]:
import json

ALGO = ['proposition', 'parent retriever', 'hybrid search']
ES_SIZE = 4910

a = {
    "QA": {"repo": QA_REPO, "split": QA_SPLIT,
           "size": qa_dataset.num_rows, 
           "total_time(minute)": round(time_np.sum() / 60, 3), 
           "median_time(second)": round(np.median(time_np), 3),
           "avg_time(second)": round(time_np.mean(), 3)},
    "RAG": {"algo": ALGO, **config, "es_size": ES_SIZE},
    "IR": data,
    "BLEU": bleu,
    "ROUGE-L": rouge
}
 
json_formatted_str = json.dumps(a, indent=2)
print(json_formatted_str)

{
  "QA": {
    "repo": "BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong",
    "split": "INDEX.medium_index_TDT",
    "size": 144,
    "total_time(minute)": 11.475,
    "median_time(second)": 4.443,
    "avg_time(second)": 4.781
  },
  "RAG": {
    "algo": [
      "proposition",
      "parent retriever",
      "hybrid search"
    ],
    "llm": "gemini-1.5-flash",
    "total_k": 4,
    "txt_weight": 0.5,
    "vec_weight": 0.5,
    "vec_index": "vec-sentence-propositon_medium_edu_tdt",
    "txt_index": "text-sentence-propositon_medium_edu_tdt",
    "es_size": 4910
  },
  "IR": {
    "k_6": {
      "precision": 0.6510416666666666,
      "recall": 0.8680555555555556,
      "map_score": 0.2829166666666666,
      "relevant_retrieved": 125,
      "num_retrieved": 192
    },
    "k_7": {
      "precision": 0.6386138613861386,
      "recall": 0.8958333333333334,
      "map_score": 0.28290426587301587,
      "relevant_retrieved": 129,
      "num_retrieved": 202
    },
    "k_8": {
      "precision": 0.